In [250]:
import pandas as pd
from sklearn import preprocessing
from datetime import datetime
from sklearn.svm import SVR

In [251]:
df = pd.read_csv("Data/ticket.csv", delimiter="\t")

#### Removing redundant sections and rows with NaN for CheckinTime

In [252]:
df.drop("TIME SECTION",1, inplace = True)
df.drop("TICKET",1, inplace = True)
df.drop("TICKET NUM",1, inplace = True)
df.drop("TABLE TYPE",1, inplace = True)
df.drop("STATUS",1, inplace = True)
df.drop("CHANNEL",1, inplace = True)

In [253]:
df.head(5)

,REST_ID,TABLE SIZE,CREATE TIME,CHECKINTIME
0,113442,5,2016-01-01 11:09:19.000,2016-01-01 11:16:41
1,113442,4,2016-01-01 11:15:45.000,2016-01-01 11:16:40
2,113442,2,2016-01-01 11:17:11.000,2016-01-01 11:17:21
3,113442,2,2016-01-01 11:34:41.000,2016-01-01 11:34:53
4,113442,5,2016-01-01 11:51:00.000,2016-01-01 12:00:36


#### Parsing the DateTime and adding columns based on it

In [254]:
dater = lambda c: datetime.strptime(c, "%Y-%m-%d %H:%M:%S.%f")
dater2 = lambda c: datetime.strptime(c, "%Y-%m-%d %H:%M:%S")

In [255]:
# df["TIME CHECKIN"] = df["CHECKINTIME"].apply(lambda a: dater2(a).hour*3600 + dater2(a).minute*60 + dater2(a).second)
df["TIME (SEC)"] = df["CREATE TIME"].apply(lambda a: dater(a).hour*3600 + dater(a).minute*60 + dater(a).second)
df["MONTH"] = df["CREATE TIME"].apply(lambda a: int(a.split()[0].split('-')[1]))
df["DATE"] = df["CREATE TIME"].apply(lambda a: dater(a).day)
df["DAY"] = df["CREATE TIME"].apply(lambda a: dater(a).isoweekday())
# df["WAIT TIME"] = df["TIME CHECKIN"] - df["TIME (SEC)"]
df["INTERVAL NUMBER"] = df["TIME (SEC)"]//1200

In [256]:
df.head(5)

,REST_ID,TABLE SIZE,CREATE TIME,CHECKINTIME,TIME (SEC),MONTH,DATE,DAY,INTERVAL NUMBER
0,113442,5,2016-01-01 11:09:19.000,2016-01-01 11:16:41,40159,1,1,5,33
1,113442,4,2016-01-01 11:15:45.000,2016-01-01 11:16:40,40545,1,1,5,33
2,113442,2,2016-01-01 11:17:11.000,2016-01-01 11:17:21,40631,1,1,5,33
3,113442,2,2016-01-01 11:34:41.000,2016-01-01 11:34:53,41681,1,1,5,34
4,113442,5,2016-01-01 11:51:00.000,2016-01-01 12:00:36,42660,1,1,5,35


#### Setting up the regression model and training on it

In [257]:
df.drop("TABLE SIZE",1, inplace = True)
df.drop("CREATE TIME",1, inplace = True)
df.drop("CHECKINTIME",1, inplace = True)
# df.drop("TIME CHECKIN",1, inplace = True)
df.drop("TIME (SEC)",1, inplace = True)
# df.drop("WAIT TIME",1, inplace = True)
df = df.groupby(df.columns.tolist()).size().reset_index().rename(columns={0:'DENSITY'})

In [258]:
df.head(5)

,REST_ID,MONTH,DATE,DAY,INTERVAL NUMBER,DENSITY
0,113442,1,1,5,33,3
1,113442,1,1,5,34,1
2,113442,1,1,5,35,4
3,113442,1,1,5,36,10
4,113442,1,1,5,37,3


In [259]:
restaurantList = df['REST_ID'].values
restaurantList = list(set(restaurantList))

In [260]:
# (((df['REST_ID'] == 113442) & (df['MONTH'] == 1) & (df['DATE'] == 1) & (df['INTERVAL NUMBER'] == 25)).any())

False

In [261]:
for r in restaurantList:
    for m in range(1,13):
        for d in range (1,29):
            for i in range (27,69):
                if (not(((df['REST_ID'] == r) & (df['MONTH'] == m) & (df['DATE'] == d) & (df['INTERVAL NUMBER'] == i)).any())):
                    day = datetime.strptime("{}-{}-{}".format(2016,m,d) , "%Y-%m-%d").isoweekday()
                    df.loc[df.shape[0]]=[r, m, d, day, i, 0] 

In [264]:
df

,REST_ID,MONTH,DATE,DAY,INTERVAL NUMBER,DENSITY
0,113442,1,1,5,33,3
1,113442,1,1,5,34,1
2,113442,1,1,5,35,4
3,113442,1,1,5,36,10
4,113442,1,1,5,37,3
5,113442,1,1,5,38,12
6,113442,1,1,5,39,12
7,113442,1,1,5,40,11
8,113442,1,1,5,41,6
9,113442,1,1,5,42,17
